# Introduction

### Project Abstract

### Figure illustrating workflow (Ruoyu)

See below for a brief summary illustration of the workflow for this project:

<img src="./img/workflow.jpg" width="1000">

# Data (Zhengyi)

# Pre-processing Pipeline (All, specific TBD)

### Functions

# Feature Spaces

### TF-IDF (Ruoyu)

### Word2Vec (Mike)

Pre-trained in library vs. fit on own document

### Extra features (Ruoyu)

# Models

### Naive Bayes (Zhengyi)

### Logistic Regression (Skylar)

### Random Forest (Mike)

### LSTM (Ruoyu, if possible)

# Result and summary

### TODO: create a table containing all accuracy scores, feature space numbers, and models (Skylar)

Columns:
- Feature 1 or 2
- Model name
- Accuracy score

# Live-prediction (Optional)